In [1]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_path = '/home/geum_bi/klue'

model = AutoModelForSequenceClassification.from_pretrained(model_path, num_labels=16).to(device)

tokenizer = AutoTokenizer.from_pretrained(model_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /home/geum_bi/klue and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
import pandas as pd

df = pd.read_csv('./mbti_labeling.csv', encoding='UTF-8')

df

,text,mbti
0,약간 로망..처럼 옥탑방에 살아보고 싶단 생각이 있는데 (몇 개월이라도) 나만의 옥...,0
1,라고 하기에 제가 가장 가까이서 본 사람이러그런 맘은 아니었다고 좋게 댓글 달아도....,0
2,"이 팁이 에프 제인 저보다 그것도 초면에 말이 많아서 제 말을 끊을 정도라니,,, ...",0
3,"저는 고등학생이라,, 현금보다는 다른 의미 있는 걸 해드리고 싶어서요 주말에 식사 ...",0
4,너무 포괄적 질문이긴 한데 한 달에 요즘 얼마 버나요?15년 전 아는 언니가 딸기농...,0
...,...,...
20936,"말하지 않아도 느껴야 했던 부분이고, 그걸 모르는 그 사람의 문제여야 하지, 참고 ...",15
20937,상황이 어떻게 돌아가든 잘 되든 못되든 고민은 누구나 다 있는 듯요. 진짜 앞으로 ...,15
20938,어릴 때부터 그랬는데 나이 들면 바뀐다 결혼하면 바뀐다 자기 애 낳으면 안 그렇다 ...,15
20939,얘기가 딴 데로 샜는데 -a로 나오는 istp하고 에 팁은 어떤지 너무 궁금해요 ...,15


In [5]:
from datasets import Dataset, DatasetDict

# Convert the DataFrame into a dictionary format suitable for Dataset creation
formatted_data = {
    'text': df['text'].tolist(),  # Extract text column
    'label': df['mbti'].tolist()  # Extract label column
}

# Create a Dataset object from the formatted data
dataset = Dataset.from_dict(formatted_data)

# Split the dataset into train and test sets (80% train, 20% test)
train_test_split = dataset.train_test_split(test_size=0.2)

# Convert the split datasets into a DatasetDict object
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'test': train_test_split['test']
})

In [6]:
dataset_dict['train'][0]

{'text': '물론, 그렇다고 이들이 안정된 직장, 안정된 지위, 안정된 관계를 마냥 거부한다는 의미는 아닙니다.그 속에서, 가치관을 존중하는 선에서 시도할 수 있는 일탈을 가끔씩은, 때로는 절실히 필요로 한다는 거죠. 아마도 새로운 사람과의 만남, 새로운 취미의 발견, 새로운 일을 향한 도전과 같은 것들이 바람직한 일탈이 되어 줄 겁니다. 결국 어제와는 뭔가 다른, 가끔은 예상치 못 한 일도 발생하는, 곳곳에 흥미로운 변화가 존재하며, 불안정 속에서 안정을, 그리고 안정 속에서 불안정을 추구하는 조금은 모순적인 삶이 이들에게는 가장 안정적인 삶인지도 모르겠습니다.',
 'label': 3}

In [7]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [8]:
tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

Map:   0%|          | 0/16752 [00:00<?, ? examples/s]

Map:   0%|          | 0/4189 [00:00<?, ? examples/s]

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',                      # 출력 디렉토리
    evaluation_strategy="epoch",                 # 매 에폭마다 평가
    learning_rate=1e-5,                          # 학습률
    per_device_train_batch_size=16,              # 학습 배치 크기
    per_device_eval_batch_size=16,               # 평가 배치 크기
    num_train_epochs=10,                         # 학습 에폭 수
    weight_decay=0.01,                           # 가중치 감쇠
    seed=42,                                     # 시드 값
    lr_scheduler_type="linear",                  # 학습률 스케줄러 타입
    warmup_ratio=0.1,                            # 워밍업 비율
    optim="adamw_torch",                         # 옵티마이저 (기본값은 AdamW)
)

2024-09-13 18:49:57.378664: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-13 18:49:57.403035: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 18:49:58.511086: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [11]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,2.679700,2.652122,0.127477
2,2.524100,2.523706,0.206254
3,2.377000,2.484131,0.226546
4,2.206900,2.493537,0.232036
5,2.010200,2.544478,0.226068


KeyboardInterrupt: 

: 

In [ ]:
model.save_pretrained('./klue-mbti')
tokenizer.save_pretrained('./klue-mbti')